In [ ]:
#### obter_dados_cards
import os
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import urllib3
from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image
import gc

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) #### Desabilitar avisos de segurança do requests


#### ___________________________________________________________________________________________________

def obter_dados_cards(setUrl, Variation, setName):
  #### Criar pasta de imagens, se não existir
  image_folder = "images"
  session2 = None  # Inicializa a variável
  session3 = None  # Inicializa a variável

  Card = ""
  Flavor = ""
  Rarity = ""
  Artist = ""
  table_div = None
  table_div_cast = None
  #### URL do card
  url = f"{setUrl}"
  data = [""] * 16  #### Inicializa a lista com 8 elementos vazios

  #### Fazer requisição HTTP
  headers = {"User-Agent": "Mozilla/5.0"}
  session2 = requests.Session()
  session3 = None
  response = session2.get(url, headers=headers, verify=False)

  if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")

    #### Encontrar variações do card
    card_variations_div = soup.find("div", class_="variations")

    if card_variations_div:
      #### Encontrar todas as variações  
      variations = card_variations_div.find_all("a")            
      qtdVar = len(variations)
      if qtdVar > 1:
        variation = variations[Variation - 1]                
        variation_url = variation["href"]
        variation_url = variation_url.replace("/Pages", "https://gatherer.wizards.com/Pages")
        #### URL do card 
        url = variation_url

        #### Fazer requisição HTTP
        headers = {"User-Agent": "Mozilla/5.0"}
        session3 = requests.Session()
        response = session3.get(url, headers=headers, verify=False)
    session2.close()

  if response.status_code == 200:
    #### Para pegar o id da Carta
    match = re.search(r"multiverseid=(\d+)", url)
    numero = match.group(1)

    # if numero == 497667 or numero == "497667":
    #   print()
    #   breakpoint()

    soup = BeautifulSoup(response.text, "html.parser")

    #### Se não tem variações
    #### Encontrar a div que contém a imagem
    #### Sem CastOff
    card_image_div = soup.find("div", class_="cardImage")
    card_image_div_castOff = soup.find("table", class_="cardDetails cardComponent")

    if card_image_div:
      img_tag = card_image_div.find("img")
    elif card_image_div_castOff:
      img_tag = card_image_div_castOff.find("img")


    if img_tag:
      img_url = img_tag["src"]
      img_url = img_url.replace("../../", "https://gatherer.wizards.com/") 
      img_alt = img_tag["alt"]
      img_name = img_alt + " - " + setName + " - " + numero + ".jpg"
      img_name = img_name.replace("_", " ")
      img_name = img_name.replace("//","&")
      img_name = img_name.replace(":"," ")
      img_name = img_name.replace("  "," ")
      
      setNameFolder = setName
      setNameFolder = setNameFolder.replace("_", " ")
      setNameFolder = setNameFolder.replace("//","&")
      setNameFolder = setNameFolder.replace(":"," ")
      setNameFolder = setNameFolder.replace("  "," ")

      image_folder = os.path.join(image_folder, setNameFolder)      
      os.makedirs(image_folder, exist_ok=True)
      img_path = os.path.join(image_folder, img_name)

      #### Baixar e salvar a imagem
      img_response = requests.get(img_url, headers=headers, verify=False, stream=True)
      if img_response.status_code == 200:
        with open(img_path, "wb") as img_file:
          for chunk in img_response.iter_content(chunk_size=8192):  # Baixa e escreve em pedaços de 8KB
            img_file.write(chunk)
      else:
        print("Erro ao baixar a imagem.")
    
    #### Sem CastOff
    table_div = soup.find("div", class_="smallGreyMono", style="margin-top: 5px;")
    if table_div:
      rows = table_div.find_all("div", class_="label")
      values = table_div.find_all("div", class_="value")
      
      for label, value in zip(rows, values):
        key = label.text.strip()
        val = value.text.strip()
        if key == "Card Text:": Card = val
        if key == "Flavor Text:": Flavor = val
        if key == "Rarity:": Rarity = val
        if key == "Artist:": Artist = val
        if key == "Card Number:": CardNumber = val
        if key == "Expansion:": Expansion = val
      
      if Card != "": data[0] = Card
      if Flavor != "": data[1] = Flavor
      if Rarity != "": data[2] = Rarity
      if Artist != "": data[3] = Artist
      if url != "": data[4] = url
      if img_path != "": data[5] = img_path
      if img_url != "": data[6] = img_url
      if img_name != "": data[7] = img_name
      if numero != "": data[8] = numero
      if CardNumber != "": data[14] = CardNumber
      if Expansion != "": data[15] = Expansion

    table_div_cast = soup.find_all("div", class_="smallGreyMono", style=None)
    counterA = 0
    for table_div in table_div_cast:
      # table_div = table_div_cast[0]
      if table_div and counterA == 0:
        rows = table_div.find_all("div", class_="label")
        values = table_div.find_all("div", class_="value")
        
        for label, value in zip(rows, values):
          key = label.text.strip()
          val = value.text.strip()
          if key == "Card Text:": Card = val
          if key == "Flavor Text:": Flavor = val
          if key == "Rarity:": Rarity = val
          if key == "Artist:": Artist = val
          if key == "Card Number:": CardNumber = val
          if key == "Expansion:": Expansion = val
        
        if Card != "": data[0] = Card
        if Flavor != "": data[1] = Flavor
        if Rarity != "": data[2] = Rarity
        if Artist != "": data[3] = Artist
        if url != "": data[4] = url
        if img_path != "": data[5] = img_path
        if img_url != "": data[6] = img_url
        if img_name != "": data[7] = img_name
        if numero != "": data[8] = numero
        if CardNumber != "": data[14] = CardNumber
        if Expansion != "": data[15] = Expansion
        
      if table_div and counterA == 1:
        rows = table_div.find_all("div", class_="label")
        values = table_div.find_all("div", class_="value")
        
        CostA = "0"
        ManaA = "0"
        TypeA = "" 
        CardA = ""
        NameA = ""  
        for label, value in zip(rows, values):
          key = label.text.strip()
          val = value.text.strip()
          if key == "Card Name:": NameA = val
          if key == "Mana Cost:":            
            cost_imgsA = value.find_all("img")
            CostA = ", ".join(img["alt"] for img in cost_imgsA)
          if key == "Mana Value:": ManaA = val
          if key == "Types:": TypeA = val
          if key == "Card Text:": CardA = val
        
        if NameA != "": data[9] = NameA
        if CostA == None: data[10] = CostA  
        if CostA != "": data[10] = CostA
        if ManaA != "": data[11] = ManaA
        if TypeA != "": data[12] = TypeA
        if CardA != "": data[13] = CardA

      counterA += 1


  else:
    print("Erro ao acessar a página.")

  if session2 == None:
    session3.close()
  else:
    session2.close()
  return data 


####___________________________________________________________________________________________


#### salvar em Excel

def salvar_em_excel(lista_cards):
  df_temp = pd.DataFrame(lista_cards, columns=columns)

  #### Se o arquivo já existe, carregar; senão, criar um novo
  if os.path.exists(excel_file):
    wb = load_workbook(excel_file)
    ws = wb.active
  else:
    wb = Workbook()
    ws = wb.active
    ws.append(columns)  #### Adicionar cabeçalho

  #### Adicionar novas linhas ao Excel e imagens
  for index, row in df_temp.iterrows():
    ws.append(row.tolist())  #### Adiciona os dados

  #### Salvar o arquivo Excel atualizado
  wb.save(excel_file) 


#### ___________________________________________________________________________________________________


#### obter_lista_cards

#### Nome do arquivo Excel
excel_file = "cards_data.xlsx"

#### Colunas do DataFrame
columns = ["Nome", "Cost", "Total Cost", "Type", "P", "T", "Edition", "Variações", #### parte que vem do site compacto
           "Card Text", "Flavor Text", "Rarity", "Artist", "Link", "Image Path", "Image URL", "Image Name", "id", #### parte que vem da pagina da carta
           "Nome_A", "Cost_A", "Total Cost_A", "Type_A", "Card Text_A", "Card Number", "Expansion"] #### parte que vem das cartas com 2 funções

#### Carregar IDs existentes, se o arquivo já existir
existing_ids = {}
if os.path.exists(excel_file):
  df_existing = pd.read_excel(excel_file, usecols=["id"])
  existing_ids = df_existing["id"].fillna(0).astype(int).tolist()  #### Converter para lista de inteiros


#----------------------------------------------------------


# Arquivo contendo a lista original de sets
sets_file = "sets.txt"

# Ler lista original e os já processados
with open(sets_file, "r", encoding="utf-8") as f:
    sets_list = [line.strip() for line in f.readlines()]

# Arquivo para armazenar os sets já processados
processed_file = "processed_sets.txt"

### Se houver sets restantes, processar o primeiro e salvar no arquivo  
while True:
  if os.path.exists(processed_file):
      with open(processed_file, "r", encoding="utf-8") as f:
          processed_list = {line.strip() for line in f.readlines()}
  else:
      processed_list = set()

  # Filtrar sets que ainda não foram processados
  remaining_sets = [set_name for set_name in sets_list if set_name not in processed_list]
    
  if not remaining_sets:  # Se a lista estiver vazia, encerramos o loop
      break
  
  setName = remaining_sets[0]  # Pega o primeiro da lista

  #### Carrega a página com a lista de cards
  page = 0
  formatted_setName = setName.replace(" ", "+")  #### Substituir espaços por '+'
  url = f"https://gatherer.wizards.com/Pages/Search/Default.aspx?sort=name+&page={page}&output=compact&set=[%22{formatted_setName}%22]"

  #### Fazer requisição para a página
  session0 = requests.Session()
  response = session0.get(url, verify=False)

  #### Se a resposta não for válida, interrompe o loop
  if response.status_code == 200:  
    soup = BeautifulSoup(response.text, "html.parser")

    #### Encontrar a div com a classe "pagincontrols"
    pagin_controls = soup.find("div", class_="pagingcontrols")
    if pagin_controls:
      links = pagin_controls.find_all("a")
      qtdPages = len(links)  #### -1 para não contar o link "Next" e começar com 0

      if qtdPages <= 1: 
        qtdPages = 1  
      else:
        qtdPages = qtdPages - 1
    session0.close()

  #### Ajuste o número de páginas conforme necessário
  for page in range(0, qtdPages):

    # Carrega a pagina com links para cada versão do card
    url = f"https://gatherer.wizards.com/Pages/Search/Default.aspx?sort=name+&page={page}&output=compact&set=[%22{formatted_setName}%22]"

    #### Refazer requisição para a página
    session1 = requests.Session()
    response = session1.get(url, verify=False)

    #### Adicionar um pequeno atraso entre as requisições      
    time.sleep(2)  

    #### Se a resposta não for válida, interrompe o loop
    if response.status_code == 200:  
      soup = BeautifulSoup(response.text, "html.parser")
      contador = 0

      print(f"\n    {setName} - página {page + 1} de {qtdPages}")
      print("123456789.123456789.123456789.123456789.123456789.123456789.123456789.123456789.123456789.1234567890123456789.123456789.123456789.123456789.123456789.")

      #### para cada linha da pagina principal
      for row in soup.find_all("tr"):
        #### Cria uma lista de 1 linha com as informações do card
        card= []

        #### executar apenas os 5 primeiros cards para testes, desabilitar para fazer o real
        contador += 1 
        # if contador >= 5: continue  #### executa apenas os 5 primeiros cards

        #### vê o conteudo de cada celula
        cols = row.find_all("td")
        if len(cols) >= 6:  #### Garantindo que há ao menos 6 colunas
          #### Extrair os dados de texto de cada coluna

          name = cols[0].text.strip()
          if name == "Name" or name == None: continue  #### Ignorar a linha de cabeçalho

          # print(f"    {setName} - página {page} - linha {contador} - {name}")

          #### Para a coluna Link, buscar o atributo href da tag <a> dentro da coluna 0
          link_tag = cols[0].find("a")
          link = link_tag["href"] if link_tag else ""
          link = link.replace("../", "https://gatherer.wizards.com/Pages/")

          # Para pegar o id da Carta
          match = re.search(r"multiverseid=(\d+)", link)
          numero = match.group(1)

          # Verifica se o ID já existe no Excel
          #if numero in existing_ids.values():
          if int(numero) in existing_ids:
            #print(f"ID {numero} já existe no Excel. Pulando...")
            print("p", end="")
            continue

          if setName == "Commander Legends" and page == 3:
            breakpoint()

          #### Extrair os dados de texto de cada coluna
          type_ = cols[2].text.strip()
          P_ = cols[3].text.strip()
          T_ = cols[4].text.strip()

          #### Para a coluna Cost, buscar todas as tags <img> e extrair o atributo "alt"
          cost_imgs = cols[1].find_all("img")
          cost = ", ".join(img["alt"] for img in cost_imgs)
          totalCost = len(cost_imgs)

          #### Para a coluna Printings, buscar todas as tags <img> e extrair o atributo "alt"
          printings_imgs = cols[5].find_all("img")
          printings = ", ".join(img["alt"] for img in printings_imgs)
          qtdImages = len(printings_imgs)

          #### Abre a pagina do Card para pegar a imagem e informações complementares
          if qtdImages > 1:
            for i in range(0, qtdImages ):
              print(i, end="")
              card= []
              dataRetorno = obter_dados_cards(link, i, setName)
              Card = dataRetorno[0]
              Flavor = dataRetorno[1]
              Rarity = dataRetorno[2]
              Artist = dataRetorno[3]
              pagina = dataRetorno[4]
              img_path = dataRetorno[5]
              img_url = dataRetorno[6]
              img_name = dataRetorno[7]
              numero = dataRetorno[8]
              nome_A = dataRetorno[9]
              cost_A = dataRetorno[10]
              totalCost_A = dataRetorno[11]
              type_A = dataRetorno[12]
              Card_Text_A = dataRetorno[13]
              Card_Number = dataRetorno[14]
              Expansion = dataRetorno[15]
              # adicionar o id à lista de ids existentes
              try:
                existing_ids.append(int(numero))
              except:
                print(f"Erro ao converter o ID {numero} para inteiro.")
                print(f"{setName} - {pagina} - {name} - {numero} - \n{link}  \n{nome_A} - {Card_Text_A}")
                continue

              #### Adicionar o card à lista
              card.append([name, cost, totalCost, type_, P_, T_, printings, i+1, 
                          Card, Flavor, Rarity, Artist, pagina, img_path, img_url, img_name, int(numero), 
                          nome_A, cost_A, totalCost_A, type_A, Card_Text_A, Card_Number, Expansion])
              salvar_em_excel(card)  #### Salvar os dados do card no Excel

          else:
            print("1", end="")
            dataRetorno = obter_dados_cards(link, 1, setName)
            Card = dataRetorno[0]
            Flavor = dataRetorno[1]
            Rarity = dataRetorno[2]
            Artist = dataRetorno[3]
            pagina = dataRetorno[4]
            img_path = dataRetorno[5]
            img_url = dataRetorno[6]
            img_name = dataRetorno[7]
            numero = dataRetorno[8]
            nome_A = dataRetorno[9]
            cost_A = dataRetorno[10]
            totalCost_A = dataRetorno[11]
            type_A = dataRetorno[12]
            Card_Text_A = dataRetorno[13]
            Card_Number = dataRetorno[14]
            Expansion = dataRetorno[15]
            # adicionar o id à lista de ids existentes
            try:
              existing_ids.append(int(numero))
            except:
              print(f"Erro ao converter o ID {numero} para inteiro.")
              print(f"{setName} - {pagina} - {name} - {numero} - \n{link}  \n{nome_A} - {Card_Text_A}")
              continue
            #### Adicionar o card à lista
            card.append([name, cost, totalCost, type_, P_, T_, printings, 1, 
                        Card, Flavor, Rarity, Artist, pagina, img_path, img_url, img_name, int(numero), 
                        nome_A, cost_A, totalCost_A, type_A, Card_Text_A, Card_Number, Expansion])
            salvar_em_excel(card)  #### Salvar os dados do card no Excel

    session1.close()  #### Fechar a sessão do requests
    gc.collect()

    # Reiniciar Python após processar um set
    print("\nReiniciando o Python")
    quit()
    os.system("MTG.ipynb")
  # fim do for page in range(0, qtdPages):

  # Registrar o set como processado
  with open(processed_file, "a", encoding="utf-8") as f:
      f.write(setName + "\n")

  #### Reiniciar Python após processar um set
  print("\nReiniciando o Python")
  quit()
  os.system("MTG.ipynb")


  remaining_sets = [set_name for set_name in sets_list if set_name not in processed_list]


print("Todos os sets foram processados!")


    Double Masters 2022 - página 1 de 4
123456789.123456789.123456789.123456789.123456789.123456789.123456789.123456789.123456789.1234567890123456789.123456789.123456789.123456789.123456789.
pppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppp
Reiniciando o Python

    Double Masters 2022 - página 2 de 4
123456789.123456789.123456789.123456789.123456789.123456789.123456789.123456789.123456789.1234567890123456789.123456789.123456789.123456789.123456789.
pppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppp
Reiniciando o Python

    Double Masters 2022 - página 3 de 4
123456789.123456789.123456789.123456789.123456789.123456789.123456789.123456789.123456789.1234567890123456789.123456789.123456789.123456789.123456789.
pppppppppppppppppppp0101010101201101210120101101201012011101101010101110101101101011110121012011010121111010101110101210110111111110120121010101201101012010101201
Reiniciando o Pyt